# NLP Code Along questions

For this code along we will build a spam filter!

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

#### load and  read the dataset,  have Spark infer the data types

In [17]:
dataset = spark.read.csv("SMSSpamCollection", sep = "\t",  header = False,inferSchema=True)
dataset = dataset.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')
dataset.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Clean and Prepare the Data

#### Create a new length feature

In [18]:
from pyspark.sql.functions import length

In [19]:
dataset = dataset.withColumn("length",length(dataset['text']))

In [20]:
dataset.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



#### print the groupy mean of class

In [21]:
dataset.groupby("class").mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In this part you transform you raw text in to tf_idf model :

- chain the transformer Tokenizer, StopWordsRemover, CountVectorizer and IDF for text to have a final column name 'tf_idf'
- use the transformer StringIndexer for class column into output column 'label'

- create feature with vector assembler 'tf_idf','length of as input columns into output column named 'features'

### use pipeline for fit and transform

In [22]:
from pyspark.ml.feature import CountVectorizer, Tokenizer, StopWordsRemover, IDF, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

In [24]:
tokenizer = Tokenizer(inputCol = 'text', outputCol = 'words')
stop_words = StopWordsRemover(inputCol = 'words', outputCol = 'stop_words')
count_vec = CountVectorizer(inputCol = 'stop_words', outputCol = 'count_vec')
tfidf = IDF(inputCol = 'count_vec', outputCol = 'tf_idf')
class_numeric = StringIndexer(inputCol = 'class', outputCol = 'label')
vec_assembler = VectorAssembler(inputCols = ['tf_idf', 'length'], outputCol = 'features')
pipeline = Pipeline(stages=[class_numeric, tokenizer, stop_words, count_vec, tfidf, vec_assembler])

train = pipeline.fit(dataset)
clean_data = train.transform(dataset)


In [25]:
clean_data.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|               words|          stop_words|           count_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,8

In [27]:
clean_data.select('label', 'features').show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



Example: it may differ for you

In [106]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13461,[8,12,33,6...|
|  0.0|(13461,[0,26,308,...|
|  1.0|(13461,[2,14,20,3...|
|  0.0|(13461,[0,73,84,1...|
|  0.0|(13461,[36,39,140...|
|  1.0|(13461,[11,57,62,...|
|  0.0|(13461,[11,55,108...|
|  0.0|(13461,[133,195,4...|
|  1.0|(13461,[1,50,124,...|
|  1.0|(13461,[0,1,14,29...|
|  0.0|(13461,[5,19,36,4...|
|  1.0|(13461,[9,18,40,9...|
|  1.0|(13461,[14,32,50,...|
|  0.0|(13461,[42,99,101...|
|  0.0|(13461,[567,1744,...|
|  1.0|(13461,[32,113,11...|
|  0.0|(13461,[86,224,37...|
|  0.0|(13461,[0,2,52,13...|
|  0.0|(13461,[0,77,107,...|
|  1.0|(13461,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



### Detect spam or Ham

now use your tf-idf data to classify spam and ham

feel free to use any classifier model

result may differ for you

In [28]:
from pyspark.ml.classification import RandomForestClassifier

In [30]:
rf= RandomForestClassifier()

In [31]:
train, test = clean_data.randomSplit([0.8, 0.2])

In [32]:
spam_model = rf.fit(train)
test_results = spam_model.transform(test)
test_results.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|class|                text|length|label|               words|          stop_words|           count_vec|              tf_idf|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  ham|&lt;#&gt;  great ...|    85|  0.0|[&lt;#&gt;, , gre...|[&lt;#&gt;, , gre...|(13423,[3,6,32,61...|(13423,[3,6,32,61...|(13424,[3,6,32,61...|[17.7802802439555...|[0.88901401219777...|       0.0|
|  ham|&lt;#&gt;  w jett...|    37|  0.0|[&lt;#&gt;, , w, ...|[&lt;#&gt;, , w, ...|(13423,[3,6,291,6...|(13423,[3,6,291,6...|(13424,[3,6,291,6...|[17.7802802439555...|[0.88901401219777...|       0.0|


In [33]:
test_results.select('label','features','rawPrediction','probability','prediction').show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[3,6,32,61...|[17.7802802439555...|[0.88901401219777...|       0.0|
|  0.0|(13424,[3,6,291,6...|[17.7802802439555...|[0.88901401219777...|       0.0|
|  0.0|(13424,[0,2,3,5,6...|[17.2480132308700...|[0.86240066154350...|       0.0|
|  0.0|(13424,[6,242,278...|[17.7802802439555...|[0.88901401219777...|       0.0|
|  0.0|(13424,[15,32,45,...|[17.7802802439555...|[0.88901401219777...|       0.0|
|  0.0|(13424,[0,195,264...|[17.7802802439555...|[0.88901401219777...|       0.0|
|  0.0|(13424,[5,67,75,2...|[17.7802802439555...|[0.88901401219777...|       0.0|
|  0.0|(13424,[2,100,542...|[17.7802802439555...|[0.88901401219777...|       0.0|
|  0.0|(13424,[4980,4996...|[17.7802802439555...|[0.88901401219777...|       0.0|
|  0.0|(13424,[8

In [110]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13461,[0,1,2,14,...|[-612.34877984332...|[0.99999999999999...|       0.0|
|  0.0|(13461,[0,1,5,15,...|[-742.97388469249...|[1.0,5.5494439698...|       0.0|
|  0.0|(13461,[0,1,6,16,...|[-1004.8197043274...|[1.0,5.0315468936...|       0.0|
|  0.0|(13461,[0,1,12,34...|[-879.22017540506...|[1.0,1.0023148863...|       0.0|
|  0.0|(13461,[0,1,15,33...|[-216.47131414494...|[1.0,1.1962236837...|       0.0|
|  0.0|(13461,[0,1,16,21...|[-673.71050817005...|[1.0,1.5549413147...|       0.0|
|  0.0|(13461,[0,1,22,26...|[-382.58333036006...|[1.0,1.7564627587...|       0.0|
|  0.0|(13461,[0,1,25,66...|[-1361.5572580867...|[1.0,2.1016772175...|       0.0|
|  0.0|(13461,[0,1,33,46...|[-378.04433557629...|[1.0,6.5844301586...|       0.0|
|  0.0|(13461,[0

### Calculate the accuracy of your model

In [41]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [42]:
evaluator = MulticlassClassificationEvaluator()

In [44]:
print("Accuracy :" + str(evaluator.evaluate(test_results, {evaluator.metricName: "accuracy"})))

Accuracy :0.8481449525452976
